In [43]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import time

pd.options.display.max_columns = None

# Pitstop comparison by team 
Throughout a race, a drive needs to routinely go to their team's pit stall for a quick maintenence, change of tires, or conduct repairs on the vehicle [Pit Stop Reference](https://statathlon.com/analysis-of-the-pit-stop-strategy-in-f1/#:~:text=In%20F1%2C%20a%20pit%20stop,connected%20to%20the%20main%20track.).  
When a driver enters the pit stall, this is referred to as a "pitstop."  
Pitstops are crucial for team success. The faster the pit crew can perform a pitstop, the faster the driver can return to the race.  
A slow pitstop can waste valuable time, and possibly cost the team their lead.  
Each F1 team has two drivers. We will be looking at the Ferrari F1 team. 
We will find the fastest pitstop time for each driver during every race. 
A pitstop will include the time it takes the driver to get to the pit, change their tires, and get back on the track.  
The pitstop time will be calculated in seconds (s). 

# F1 2023 Season
The 2023 Season started in Bahrain on 2023-03-05.  
The 2023 Season ended in Abu Dhabi on 2023-11-26. 
There are a total of 23 rounds or race events. 

In [44]:
# Pit stop time by team and race (reference back using raceId and DriverId)
# Starting df, will merge information from other 
pit_stops_df = pd.read_csv('f1db_csv/pit_stops.csv', index_col=False) # pitstop data

drivers_df = pd.read_csv('f1db_csv/drivers.csv', index_col=False) # F1 Drivers 

constructors_df = pd.read_csv('f1db_csv/constructors.csv', index_col=False) # F1 teams

races_df = pd.read_csv('f1db_csv/races.csv', index_col=False) # F1 Races

circuit_df = pd.read_csv('f1db_csv/circuits.csv', index_col=False) # F1 Tracks

results_df = pd.read_csv('f1db_csv/results.csv', index_col=False)

In [45]:
pit_stops_df = pit_stops_df.drop(columns=['time'])
pit_stops_df = pit_stops_df.rename(columns={'duration': 'seconds'})
pit_stops_df

,raceId,driverId,stop,lap,seconds,milliseconds
0,841,153,1,1,26.898,26898
1,841,30,1,1,25.021,25021
2,841,17,1,11,23.426,23426
3,841,4,1,12,23.251,23251
4,841,13,1,13,23.842,23842
...,...,...,...,...,...,...
10853,1128,840,48,3,28.211,28211
10854,1128,1,51,2,24.232,24232
10855,1128,830,52,2,23.813,23813
10856,1128,858,57,2,24.384,24384


In [46]:
# drivers df cleaning
drivers_df = drivers_df[['driverId', 'number', 'forename', 'surname', 'dob']]
drivers_df = drivers_df.rename(columns={'forename': 'firstname', 'surname': 'lastname'})
drivers_df

,driverId,number,firstname,lastname,dob
0,1,44,Lewis,Hamilton,1985-01-07
1,2,\N,Nick,Heidfeld,1977-05-10
2,3,6,Nico,Rosberg,1985-06-27
3,4,14,Fernando,Alonso,1981-07-29
4,5,\N,Heikki,Kovalainen,1981-10-19
...,...,...,...,...,...
854,856,21,Nyck,de Vries,1995-02-06
855,857,81,Oscar,Piastri,2001-04-06
856,858,2,Logan,Sargeant,2000-12-31
857,859,40,Liam,Lawson,2002-02-11


In [47]:
constructors_df = constructors_df[['constructorId', 'name']]
constructors_df = constructors_df.rename(columns={'name': 'teamname'})
constructors_df

,constructorId,teamname
0,1,McLaren
1,2,BMW Sauber
2,3,Williams
3,4,Renault
4,5,Toro Rosso
...,...,...
207,210,Haas F1 Team
208,211,Racing Point
209,213,AlphaTauri
210,214,Alpine F1 Team


In [48]:
# races df cleaning
races_df = races_df.loc[races_df['year'] == 2023].reset_index(drop=True) # only grabbing races during 2023
races_df = races_df[['raceId', 'year', 'round', 'circuitId', 'name', 'date']]
races_df = races_df.rename(columns={'name': 'gpname'}) # set up grand prix name
races_df

,raceId,year,round,circuitId,gpname,date
0,1098,2023,1,3,Bahrain Grand Prix,2023-03-05
1,1099,2023,2,77,Saudi Arabian Grand Prix,2023-03-19
2,1100,2023,3,1,Australian Grand Prix,2023-04-02
3,1101,2023,4,73,Azerbaijan Grand Prix,2023-04-30
4,1102,2023,5,79,Miami Grand Prix,2023-05-07
5,1104,2023,6,6,Monaco Grand Prix,2023-05-28
6,1105,2023,7,4,Spanish Grand Prix,2023-06-04
7,1106,2023,8,7,Canadian Grand Prix,2023-06-18
8,1107,2023,9,70,Austrian Grand Prix,2023-07-02
9,1108,2023,10,9,British Grand Prix,2023-07-09


In [49]:
# we probably don't need this dataframe until the end
## Nice info 
circuit_df = circuit_df[['circuitId', 'circuitRef', 'name', 'location', 'country']]
circuit_df = circuit_df.rename(columns={'name': 'gpname'}) # set up grand prix name
circuit_df

,circuitId,circuitRef,gpname,location,country
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain
4,5,istanbul,Istanbul Park,Istanbul,Turkey
...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia
75,78,losail,Losail International Circuit,Al Daayen,Qatar


# Combining dataframes to get the necessary information:
1. Driver Information by Team
1. Team/Constructor Information
1. Pit Stop Time
1. Sort by Race and Circuit (Race Track)

In [50]:
# all races in 2023
races_df

,raceId,year,round,circuitId,gpname,date
0,1098,2023,1,3,Bahrain Grand Prix,2023-03-05
1,1099,2023,2,77,Saudi Arabian Grand Prix,2023-03-19
2,1100,2023,3,1,Australian Grand Prix,2023-04-02
3,1101,2023,4,73,Azerbaijan Grand Prix,2023-04-30
4,1102,2023,5,79,Miami Grand Prix,2023-05-07
5,1104,2023,6,6,Monaco Grand Prix,2023-05-28
6,1105,2023,7,4,Spanish Grand Prix,2023-06-04
7,1106,2023,8,7,Canadian Grand Prix,2023-06-18
8,1107,2023,9,70,Austrian Grand Prix,2023-07-02
9,1108,2023,10,9,British Grand Prix,2023-07-09


In [51]:
# using the results dataframe because it uses the Driver ID & Constructor (Team) ID
results_df = results_df[['raceId', 'driverId', 'constructorId']]
results_df = results_df.drop_duplicates().reset_index(drop=True)
results_df

,raceId,driverId,constructorId
0,18,1,1
1,18,2,2
2,18,3,3
3,18,4,4
4,18,5,1
...,...,...,...
26380,1129,832,6
26381,1129,848,3
26382,1129,815,9
26383,1129,844,6


In [52]:
missing_driver_missing_team_ps = pd.merge(pit_stops_df, results_df, on=['raceId', 'driverId'], how='inner')
missing_driver_missing_team_ps

,raceId,driverId,stop,lap,seconds,milliseconds,constructorId
0,841,153,1,1,26.898,26898,5
1,841,153,2,17,24.463,24463,5
2,841,153,3,35,26.348,26348,5
3,841,30,1,1,25.021,25021,131
4,841,30,2,17,23.988,23988,131
...,...,...,...,...,...,...,...
10853,1128,822,1,1,39:54.053,2394053,15
10854,1128,822,15,2,24.239,24239,15
10855,1128,832,1,1,38:47.072,2327072,6
10856,1128,855,1,1,39:46.414,2386414,15


In [53]:
driver_missing_team_ps = pd.merge(missing_driver_missing_team_ps, drivers_df, on=['driverId'], how='inner')
driver_missing_team_ps

,raceId,driverId,stop,lap,seconds,milliseconds,constructorId,number,firstname,lastname,dob
0,841,153,1,1,26.898,26898,5,\N,Jaime,Alguersuari,1990-03-23
1,841,153,2,17,24.463,24463,5,\N,Jaime,Alguersuari,1990-03-23
2,841,153,3,35,26.348,26348,5,\N,Jaime,Alguersuari,1990-03-23
3,842,153,1,16,23.584,23584,5,\N,Jaime,Alguersuari,1990-03-23
4,842,153,2,32,22.888,22888,5,\N,Jaime,Alguersuari,1990-03-23
...,...,...,...,...,...,...,...,...,...,...,...
10853,1114,859,2,25,23.671,23671,213,40,Liam,Lawson,2002-02-11
10854,1115,859,1,3,29.330,29330,213,40,Liam,Lawson,2002-02-11
10855,1115,859,2,21,28.665,28665,213,40,Liam,Lawson,2002-02-11
10856,1115,859,3,39,28.171,28171,213,40,Liam,Lawson,2002-02-11


In [54]:
df_driver_team_ps = pd.merge(driver_missing_team_ps, constructors_df, on=['constructorId'], how='inner')
df_driver_team_ps

,raceId,driverId,stop,lap,seconds,milliseconds,constructorId,number,firstname,lastname,dob,teamname
0,841,153,1,1,26.898,26898,5,\N,Jaime,Alguersuari,1990-03-23,Toro Rosso
1,841,153,2,17,24.463,24463,5,\N,Jaime,Alguersuari,1990-03-23,Toro Rosso
2,841,153,3,35,26.348,26348,5,\N,Jaime,Alguersuari,1990-03-23,Toro Rosso
3,842,153,1,16,23.584,23584,5,\N,Jaime,Alguersuari,1990-03-23,Toro Rosso
4,842,153,2,32,22.888,22888,5,\N,Jaime,Alguersuari,1990-03-23,Toro Rosso
...,...,...,...,...,...,...,...,...,...,...,...,...
10853,966,839,1,15,23.866,23866,209,31,Esteban,Ocon,1996-09-17,Manor Marussia
10854,967,839,1,20,33:29.052,2009052,209,31,Esteban,Ocon,1996-09-17,Manor Marussia
10855,967,839,2,28,24:45.092,1485092,209,31,Esteban,Ocon,1996-09-17,Manor Marussia
10856,968,839,1,22,24.035,24035,209,31,Esteban,Ocon,1996-09-17,Manor Marussia


In [55]:
df_driver_team_race_ps = pd.merge(df_driver_team_ps, races_df, on=['raceId'], how='inner')
df_driver_team_race_ps

,raceId,driverId,stop,lap,seconds,milliseconds,constructorId,number,firstname,lastname,dob,teamname,year,round,circuitId,gpname,date
0,1098,1,1,12,24.682,24682,131,44,Lewis,Hamilton,1985-01-07,Mercedes,2023,1,3,Bahrain Grand Prix,2023-03-05
1,1098,1,2,30,24.690,24690,131,44,Lewis,Hamilton,1985-01-07,Mercedes,2023,1,3,Bahrain Grand Prix,2023-03-05
2,1098,847,1,13,27.062,27062,131,63,George,Russell,1998-02-15,Mercedes,2023,1,3,Bahrain Grand Prix,2023-03-05
3,1098,847,2,31,24.406,24406,131,63,George,Russell,1998-02-15,Mercedes,2023,1,3,Bahrain Grand Prix,2023-03-05
4,1098,815,1,17,24.264,24264,9,11,Sergio,Pérez,1990-01-26,Red Bull,2023,1,3,Bahrain Grand Prix,2023-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,1115,807,2,27,28.274,28274,210,27,Nico,Hülkenberg,1987-08-19,Haas F1 Team,2023,17,78,Qatar Grand Prix,2023-10-08
908,1115,807,3,39,28.591,28591,210,27,Nico,Hülkenberg,1987-08-19,Haas F1 Team,2023,17,78,Qatar Grand Prix,2023-10-08
909,1115,825,1,3,28.743,28743,210,20,Kevin,Magnussen,1992-10-05,Haas F1 Team,2023,17,78,Qatar Grand Prix,2023-10-08
910,1115,825,2,21,28.948,28948,210,20,Kevin,Magnussen,1992-10-05,Haas F1 Team,2023,17,78,Qatar Grand Prix,2023-10-08


In [56]:
df_ferrari_ps = df_driver_team_race_ps.loc[df_driver_team_race_ps['teamname'] == 'Ferrari'].reset_index(drop=True)
df_ferrari_ps.columns = df_ferrari_ps.columns.str.strip() # to remove whitespace in headers

df_ferrari_ps['firstname'] = df_ferrari_ps['firstname'].str.strip()
df_ferrari_ps['lastname'] = df_ferrari_ps['lastname'].str.strip()
df_ferrari_ps['teamname'] = df_ferrari_ps['teamname'].str.strip()
df_ferrari_ps['gpname'] = df_ferrari_ps['gpname'].str.strip()

df_ferrari_ps = df_ferrari_ps.replace(np.nan, '')

In [57]:
df_ferrari_ps

,raceId,driverId,stop,lap,seconds,milliseconds,constructorId,number,firstname,lastname,dob,teamname,year,round,circuitId,gpname,date
0,1098,832,1,13,24.500,24500,6,55,Carlos,Sainz,1994-09-01,Ferrari,2023,1,3,Bahrain Grand Prix,2023-03-05
1,1098,832,2,31,24.227,24227,6,55,Carlos,Sainz,1994-09-01,Ferrari,2023,1,3,Bahrain Grand Prix,2023-03-05
2,1098,844,1,13,24.345,24345,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,1,3,Bahrain Grand Prix,2023-03-05
3,1098,844,2,33,24.644,24644,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,1,3,Bahrain Grand Prix,2023-03-05
4,1099,832,1,15,20.760,20760,6,55,Carlos,Sainz,1994-09-01,Ferrari,2023,2,77,Saudi Arabian Grand Prix,2023-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1120,844,1,17,21.919,21919,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,22,24,Abu Dhabi Grand Prix,2023-11-26
72,1120,844,2,35,21.246,21246,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,22,24,Abu Dhabi Grand Prix,2023-11-26
73,1115,844,1,12,28.228,28228,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,17,78,Qatar Grand Prix,2023-10-08
74,1115,844,2,25,27.873,27873,6,16,Charles,Leclerc,1997-10-16,Ferrari,2023,17,78,Qatar Grand Prix,2023-10-08


In [58]:
gp_list = df_ferrari_ps['gpname'].unique().tolist()
gp_list

['Bahrain Grand Prix',
 'Saudi Arabian Grand Prix',
 'Australian Grand Prix',
 'Azerbaijan Grand Prix',
 'Miami Grand Prix',
 'Monaco Grand Prix',
 'Spanish Grand Prix',
 'Canadian Grand Prix',
 'Austrian Grand Prix',
 'British Grand Prix',
 'Hungarian Grand Prix',
 'Belgian Grand Prix',
 'Dutch Grand Prix',
 'Italian Grand Prix',
 'Singapore Grand Prix',
 'Japanese Grand Prix',
 'United States Grand Prix',
 'Mexico City Grand Prix',
 'São Paulo Grand Prix',
 'Las Vegas Grand Prix',
 'Abu Dhabi Grand Prix',
 'Qatar Grand Prix']

In [59]:
for gp in gp_list:
    df_gp_carlos = df_ferrari_ps.loc[(df_ferrari_ps['gpname'] == f'{gp}') & (df_ferrari_ps['firstname'] == 'Carlos')].reset_index(drop=True)
    df_gp_charles = df_ferrari_ps.loc[(df_ferrari_ps['gpname'] == f'{gp}') & (df_ferrari_ps['firstname'] == 'Charles')].reset_index(drop=True)
    pit_time_carlos = df_gp_carlos['seconds'].min()
    pit_time_charles = df_gp_charles['seconds'].min()
    print(f'Carlos fastest pit time in {gp} is {pit_time_carlos}')
    print(f'Charles fastest pit time in {gp} is {pit_time_charles}')

Carlos fastest pit time in Bahrain Grand Prix is 24.227
Charles fastest pit time in Bahrain Grand Prix is 24.345
Carlos fastest pit time in Saudi Arabian Grand Prix is 20.760
Charles fastest pit time in Saudi Arabian Grand Prix is 20.026
Carlos fastest pit time in Australian Grand Prix is 14:34.924
Charles fastest pit time in Australian Grand Prix is nan
Carlos fastest pit time in Azerbaijan Grand Prix is 21.970
Charles fastest pit time in Azerbaijan Grand Prix is 21.126
Carlos fastest pit time in Miami Grand Prix is 22.278
Charles fastest pit time in Miami Grand Prix is 22.068
Carlos fastest pit time in Monaco Grand Prix is 24.252
Charles fastest pit time in Monaco Grand Prix is 24.192
Carlos fastest pit time in Spanish Grand Prix is 22.321
Charles fastest pit time in Spanish Grand Prix is 21.778
Carlos fastest pit time in Canadian Grand Prix is 24.133
Charles fastest pit time in Canadian Grand Prix is 24.343
Carlos fastest pit time in Austrian Grand Prix is 16.380
Charles fastest pit